In [ ]:
%%capture 
!apt install gdal-bin python-gdal python3-gdal 
%pip install pandas fiona shapely pyproj
!apt install python3-rtree 
!pip install geopandas==0.10.0
!pip install descartes
!pip install scipy 
!pip install statsmodels
!pip install matplotlib
!pip install gdal
!pip install numpy
!pip install scikit-learn
!pip install statsmodels

In [ ]:
#Here we are importing the packages we need. 
import geopandas as gpd
import pandas as pd 
from geopandas.tools import sjoin
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import shape
from descartes import PolygonPatch
import time
import math
import scipy.stats as stats
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import matplotlib as mpl
from math import floor

from shapely.ops import unary_union

import warnings
warnings.filterwarnings('ignore')

from osgeo import ogr, gdal,osr

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def get_val_in_fire_canlabs(fire_shapefile, shapefile,data_surface,\
                            transform,size,srcds,year_fire,fire_years_raster,\
                            data_pre,factor,threshold=0.8):
    '''This is a function to get the value inside the fire.
    We will use to calculate the mean, median, max value for a fire.
    
    Parameters
    ----------
        fire_shapefile : string
            path to the fire shapefile 
        shapefile : string
            path to the study area shapefile
        data_surface : ndarray
            an array of values in the study area
        transform : list 
            list describing GeoTransform of raster 
        size : list 
            pixel dimensions
        srcds : GDAL object
            read in raster
        year_fire : int
            year of fire from CNFDB
        fire_years_raster : ndarray 
            fire years raster from canlabs in ndarray format
        factor : int
            factor by which to multiply number of points in each pixel 
            higher = more accurate 
        threshold : float
            how much cloud mask to tolerate      
            by default it is 0.8 
            
    Returns
    ----------
        float
            - maximum value in fire, either the closest point to the convex hull of the fire or a sum of the points inside the fire
    '''

    #na_map = gpd.read_file(shapefile)
    #bounds = shapefile.bounds  # Get the bounding box of the shapefile
    fire_years_raster = np.flipud(fire_years_raster)
    data_surface = np.flipud(data_surface)
    data_pre = np.flipud(data_pre)
    bounds = gpd.GeoDataFrame(geometry=[fire_shapefile['geometry']]).bounds 
    xmax = np.nanmax(bounds['maxx'])
    xmin = np.nanmin(bounds['minx'])
    ymax = np.nanmax(bounds['maxy'])
    ymin = np.nanmin(bounds['miny'])

    xOrigin = transform[0]
    yOrigin = transform[3]
    xMax = xOrigin + transform[1] * size[0]
    yMin = yOrigin + transform[5] * size[1]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    # Calculate the number of rows cols to fill the bounding box at that resolution
    num_col = int((xmax - xmin) / pixelHeight)+1
    num_row = int((ymax - ymin) / pixelWidth)+1

    # Add the bounding box coords to the dataset so we can extrapolate the interpolation to cover whole area
    yProj_extent = [bounds['maxy'], bounds['miny']]
    xProj_extent = [bounds['maxx'], bounds['minx']]

    # Get the value for lat lon in each cell we just made
    Yi = np.linspace(np.min(yProj_extent), np.max(yProj_extent), num_row*factor)
    Xi = np.linspace(np.min(xProj_extent), np.max(xProj_extent), num_col*factor)

    Xi, Yi = np.meshgrid(Xi, Yi)
    # Because we are not using the lookup file, send in X,Y order
    concat = np.array((Xi.flatten(), Yi.flatten())).T
    send_to_list = concat.tolist()

    meshPoints = [Point(item) for item in send_to_list]
    #df = pd.DataFrame(meshPoints)
    gdf = gpd.GeoDataFrame(geometry=meshPoints)

    DF = fire_shapefile
    try: #If there is a single polygon in the shapefile
        DF = unary_union(Polygon(DF['geometry'])) #Multipolygon --> Polygon
        poly_define = gpd.GeoDataFrame(geometry=[DF])
        # Get points falling in fire 
        within_fire = gdf[gdf.geometry.within(poly_define['geometry'][0])]
    # Catch the case where multiple polygons not continuous make up the shp
    except (NotImplementedError,AttributeError,TypeError) as e:
        DF = [unary_union(Polygon(geom)) for geom in list(DF['geometry'])]
        poly_define = gpd.GeoDataFrame(geometry=DF)
        # Left spatial join 
        within_fire = sjoin(gdf, poly_define, how='left',op='within')
        # Drop points that are not in the fire 
        within_fire = within_fire[~np.isnan(within_fire['index_right'])]


    inside_fire = []
    years_inside_fire = [] 
    inside_fire_pre = [] 
    lon = []
    lat = [] 
    listP = within_fire

    for idx,p in listP.iterrows():
        mx,my=np.array(p['geometry'].coords.xy[0])[0], np.array(p['geometry'].coords.xy[1])[0]
        col = int(np.floor((mx - xOrigin)) / pixelHeight)
        row = int(np.floor((my - yOrigin)) / pixelWidth)
        
        sev = data_surface[row-1][col]
            
        if sev != -32768: #No data value
            inside_fire.append(data_surface[row-1][col])
            inside_fire_pre.append(data_pre[row-1][col])
            years_inside_fire.append(int(fire_years_raster[row-1][col]))
            
        else:
            inside_fire.append(np.nan)
            inside_fire_pre.append(np.nan)
            years_inside_fire.append(np.nan)
        
        lon.append(mx)
        lat.append(my)
        
    
    within_fire['dNBR'] = inside_fire
    within_fire['pre'] = inside_fire_pre
    within_fire['lon'] = lon
    within_fire['lat'] = lat
    within_fire['years'] = years_inside_fire

    #Calculate RBR
    within_fire['process_step'] = (within_fire['pre']/1000)+1.001
    within_fire['RBR'] = within_fire['dNBR']/within_fire['process_step']

    #within_fire = within_fire[within_fire['years'] == year_fire]
    #print(within_fire[~np.isnan(within_fire['years'])])
    #print(within_fire[~np.isnan(within_fire['max_val'])])

    
    #fig, ax = plt.subplots(figsize=(15, 15))
   # gk = plt.imshow(data_surface,extent=(
                #xOrigin,xMax,yOrigin,
                #yMin),vmin=0,vmax=2015,cmap='Spectral')
        
    #sc = plt.scatter(mx,my,
                        # c=fire_years_raster[row-1][col],cmap='Spectral',edgecolors='k',
                         #vmin=0,vmax=2015,s=38)
   # gpd.GeoDataFrame(geometry=
                        # [fire_shapefile['geometry']]).plot(ax=ax,
                                                           # facecolor='None',
                                                           # edgecolor='k')
   # c = fig.colorbar(gk)
   # c.set_label('% Basal Area Pinus Banksiana', rotation=270)
   # ax.set_title(fire_shapefile['FIRE_ID'])
   # ax.set_ylim([my-50000,
                    # my+50000])
   # ax.set_yticks([])
   # ax.set_xlim([mx-50000,
                    # mx+50000])
   # ax.set_xticks([])
   # plt.show()


    if year_fire in list(within_fire['years']): 

        if len(within_fire) > 0:
            numnan = len(within_fire[~np.isnan(within_fire['RBR'])]) / len(within_fire['RBR'])
            #print(numnan)

            if numnan >= threshold:
                mean = np.nanmean(list(within_fire['RBR']))  # get the mean val inside the fire
                median = np.nanmedian(list(within_fire['RBR']))
                maximum = np.nanmax(list(within_fire['RBR']))

            else:
                mean = np.nan
                median = np.nan
                maximum = np.nan
                
        else:
            mean = np.nan
            median = np.nan
            maximum = np.nan
    else: 
        mean = np.nan
        median = np.nan
        maximum = np.nan      

    return mean,median,maximum   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Navigate to folder where data is stored in the drive. 
%cd /content/drive/MyDrive/jameslab/

dirname = '/content/drive/MyDrive/jameslab/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/jameslab


In [ ]:
sa = gpd.read_file('study_area/BC_AB_shapefile.shp')
shp = gpd.read_file('fire_perimeters/NFDB_cipped_epsg4326_2.shp')
shp = shp[shp['SIZE_HA'] >= 10]
shp_prov = shp[shp['SRC_AGENCY'].isin(['BC'])]
shp_year = shp_prov[shp_prov['YEAR'] >= 1985]
shp_year = shp_year[shp_year['YEAR'] <= 2015]
shp_type = shp_year[shp_year['CAUSE'] == 'L'].to_crs('esri:102001')
print(len(shp_type))

fire_sev_list = []
y_ds1 = gdal.Open('canlabs/years_BC_AB_500m_align3.tif')
yb2=y_ds1.GetRasterBand(1)
ycols = y_ds1.RasterXSize
yrows = y_ds1.RasterYSize
ydata = yb2.ReadAsArray(0, 0, ycols, yrows)

src_ds1 = gdal.Open('canlabs/canlabs_BC_AB_500m_dNBR.tif')
rb2=src_ds1.GetRasterBand(1)
transform=src_ds1.GetGeoTransform()
cols = src_ds1.RasterXSize
rows = src_ds1.RasterYSize

data = rb2.ReadAsArray(0, 0, cols, rows)

src_ds3 = gdal.Open('canlabs/canlabs_BC_AB_500m.tif')
rb3=src_ds3.GetRasterBand(1)
cols = src_ds1.RasterXSize #same
rows = src_ds1.RasterYSize

predata = rb3.ReadAsArray(0, 0, cols, rows)

for index, fire in shp_type.iterrows():
    #print(fire['FIRE_ID'])
    year = int(fire['YEAR'])

    ave,med,max_val = get_val_in_fire_canlabs(fire, sa,data,transform,\
                                              (cols,rows,),src_ds1,year,\
                                              ydata,predata,1,threshold=0.8)
    fire_sev_list.append([ave,med,max_val])

shp_type['fire_severity_mean'] = [x[0] for x in fire_sev_list]
shp_type['fire_severity_median'] = [x[1] for x in fire_sev_list]
shp_type['fire_severity_max'] = [x[2] for x in fire_sev_list]
print(shp_type)

shp_type.to_csv('fire_severity_in_cnfdb_fires6.txt',sep=',')

1481
      SRC_AGENCY      FIRE_ID  ... fire_severity_median  fire_severity_max
0             BC  2004-G30401  ...                  NaN                NaN
1             BC  2004-G30485  ...                  NaN                NaN
2             BC  2004-G90441  ...                  NaN                NaN
3             BC  2004-K10384  ...                  NaN                NaN
4             BC  2004-K10398  ...           488.839286         572.335025
...          ...          ...  ...                  ...                ...
26310         BC  2015-R50101  ...                  NaN                NaN
26315         BC  2015-G60353  ...                  NaN                NaN
26319         BC  2015-V10247  ...                  NaN                NaN
26330         BC  2015-R90088  ...                  NaN                NaN
26331         BC  2015-R90047  ...                  NaN                NaN

[1481 rows x 30 columns]


In [ ]:
shp_type = shp_type[~np.isnan(shp_type['fire_severity_median'])]
print(shp_type[['FIRE_ID',\
          'fire_severity_mean','fire_severity_max','fire_severity_median']])
print('Max value detected')
print(max(shp_type['fire_severity_max']))
print('Min value detected')
print(min(shp_type['fire_severity_median']))
print(min(shp_type['fire_severity_mean']))
print(min([min(shp_type['fire_severity_mean']),min(shp_type['fire_severity_median'])]))
shp_type[['SRC_AGENCY','FIRE_ID','FIRENAME','YEAR','MONTH','DAY','REP_DATE',\
          'DATE_TYPE','OUT_DATE','DECADE','SIZE_HA','CALC_HA','CAUSE','ACQ_DATE',\
          'fire_severity_mean','fire_severity_max','fire_severity_median']].to_csv('fire_severity_in_cnfdb_fires_nonnan5.txt',sep=',')
shp_type= gpd.GeoDataFrame(shp_type,crs='esri:102001',geometry=shp_type['geometry'])
shp_type .to_file(driver = 'ESRI Shapefile',filename='fire_severity_selected')

           FIRE_ID  fire_severity_mean  fire_severity_max  fire_severity_median
4      2004-K10398          348.463827         572.335025            488.839286
10     2004-N40379          140.133515         303.561644            140.133515
11     2004-N70177          300.859493         438.481675            300.859493
18     2006-K21065          -22.570592         -22.570592            -22.570592
58     2006-N70278          327.664266         451.432565            342.037890
...            ...                 ...                ...                   ...
25831  2013-R10252          398.850637         641.180012            417.884278
25926  2014-V10244          303.789338         303.789338            303.789338
26082  2014-K70430          462.906193         462.906193            462.906193
26090  2015-G90175          336.401449         885.652643            375.973578
26097  2015-G90243          182.665930         394.219653            179.586028

[113 rows x 4 columns]
Max value detect

In [ ]:
a = shp_type[shp_type['fire_severity_max'] == 1086.913086913087]
print(a[['SIZE_HA','REP_DATE','fire_severity_median']]) 

       SIZE_HA    REP_DATE  fire_severity_median
13577   2758.5  2009-07-29            470.225873
